# ENGRI 1120: Parallel Cell-free Production and Recovery of the mRNA BNT-162b2 Vaccine

#### Your names and netids go here

<img src="figs/Fig-Parallel-System.png" style="width:50%">

## Introduction
The project introduction goes here. Typically 2-3 paragraphs.

__Suggestions__:
* Paragraph 1: Describe the motivation and background information on the product and the vaccine 
* Paragraph 2/3: Summary of your project. Describe the number of units (don't forget the heat exchangers), details about the units’ operation, and your project’s financials. 

You can replace the figure above with your system.

## Materials and Methods

### Project Setup

In [1]:
import Pkg; Pkg.activate("."); Pkg.resolve(); Pkg.instantiate();

  Activating project at `~/Desktop/julia_work/ENGRI-1120-IntroToChemE-Example-Notebooks/project`
  No Changes to `~/Desktop/julia_work/ENGRI-1120-IntroToChemE-Example-Notebooks/project/Project.toml`
  No Changes to `~/Desktop/julia_work/ENGRI-1120-IntroToChemE-Example-Notebooks/project/Manifest.toml`


In [2]:
# load reqd packages and set paths -
using JLD2
using FileIO
using PrettyTables
using DataFrames
using GLPK

# setup paths -
const _ROOT = pwd();
const _PATH_TO_DATA = joinpath(_ROOT, "data");

#### Load the project code library
The call to the `include` function loads the `ENGRI-1120-Project-CodeLib.jl` library into the notebook; the library contains functions we can use during the project. In particular, it includes the function:

* The `compute_optimal_extent(stoichiometric_matrix::Array{Float64,2}, default_bounds_array::Array{Float64,2},
    species_bounds_array::Array{Float64,2}, objective_coefficient_array::Array{Float64,1}; min_flag::Bool = true) -> Tuple` function calls the [GLPK](https://www.gnu.org/software/glpk/) linear program solver. The `results` tuple contains several things, but the important ones are `calculated_flux_array`, `objective_value`, and the status/exit flags `status_flag` and `exit_flag` (which let us know if the solver successfully found a solution).
* The `build(model::Type{MSULatticeModel}; ṅₒ::Float64, L::Int64, u::Float64, d::Float64) -> MSULatticeModel` function builds a [Binary tree](https://en.wikipedia.org/wiki/Binary_tree) of Magical Separation Units (MSUs). Arguments: $ṅₒ$ denotes the species mole flow rate into the separation system, $L$ denotes the number of layers of the tree, $u$ denotes the `up` factor (split for the `up` path), and $d$ denotes the `down` factor (the split for the `down` path). This function returns the `MSULatticeModel` model, which contains the column array `data` holding the species mole flow rate for each node in the tree. 
* The `build_nodes_dictionary(levels::Int64) -> Dict{Int64,Array{Int64,1}}` function constructs a dictionary of node indexes for each level of the tree; keys are the tree levels.

In [3]:
include("ENGRI-1120-Project-CodeLib.jl");

In [4]:
# load the model file -
model = load(joinpath(_PATH_TO_DATA, "ENGRI-1120-BNT162b2-Model.jld2"))["model"]

Dict{String, Any} with 7 entries:
  "stochiometric_matrix" => [-1.0 0.0 … 0.0 -1.0; -1.0 0.0 … -1.0 0.0; … ; 1.0 …
  "list_of_reactions"    => ["TX_BNT_162b2_binding", "TX_BNT_162b2_open", "BNT_…
  "reaction_table"       => 6×7 DataFrame…
  "flux_bounds_array"    => [-1000.0 1000.0; 0.0 1000.0; … ; 0.0 1000.0; 0.0 10…
  "mRNA_sequence"        => ['C', 'U', 'C', 'U', 'U', 'A', 'U', 'U', 'U', 'G'  …
  "list_of_species"      => ["G_BNT_162b2", "T7RNAP", "M_atp_c", "M_utp_c", "M_…
  "gene_sequence"        => ['G', 'A', 'G', 'A', 'A', 'T', 'A', 'A', 'A', 'C'  …

In [5]:
# get stuff from the model data structure -
S = model["stochiometric_matrix"]; # fix the spelling in the model file
flux_bounds_array = model["flux_bounds_array"];
list_of_species = model["list_of_species"];
list_of_reactions = model["list_of_reactions"];
reaction_table = model["reaction_table"];
gene_sequence = model["gene_sequence"];

In [6]:
reaction_table

Row,id,forward,reverse,reversibility,LB,UB,ec
,String,String,String,Bool,Float64?,Float64?,String?
1,TX_BNT_162b2_binding,G_BNT_162b2+T7RNAP,G_BNT_162b2_T7RNAP_closed,true,-Inf,Inf,missing
2,TX_BNT_162b2_open,G_BNT_162b2_T7RNAP_closed,G_BNT_162b2_T7RNAP_open,false,0.0,Inf,missing
3,BNT_162b2_transcription,G_BNT_162b2_T7RNAP_open+798*M_atp_c+1004*M_utp_c+1060*M_ctp_c+1312*M_gtp_c,mRNA_BNT_162b2+G_BNT_162b2+T7RNAP+4174*M_ppi_c,false,0.0,Inf,2.7.7.6
4,mRNA_BNT_162b2_degradation,mRNA_BNT_162b2,798*M_amp_c+1004*M_ump_c+1060*M_cmp_c+1312*M_gmp_c,false,0.0,Inf,missing
5,RNAP_deactivation,T7RNAP,T7RNAP_inactive,false,0.0,Inf,missing
6,GENE_deactivation,G_BNT_162b2,G_BNT_162b2_inactive,false,0.0,Inf,missing


In [7]:
# How many species and reactions?
(ℳ, ℛ) = size(S);

In [8]:
# initialize -
species_index_table_data = Array{Any,2}(undef, ℳ, 2);

# build table -
for i ∈ 1:ℳ
    species_index_table_data[i,1] = i;
    species_index_table_data[i,2] = list_of_species[i];
end

# setup header -
species_index_header_table = (["Index", "Species"]);

# build table -
pretty_table(species_index_table_data; header=species_index_header_table);

┌───────┬───────────────────────────┐
│ Index │                   Species │
├───────┼───────────────────────────┤
│     1 │               G_BNT_162b2 │
│     2 │                    T7RNAP │
│     3 │                   M_atp_c │
│     4 │                   M_utp_c │
│     5 │                   M_ctp_c │
│     6 │                   M_gtp_c │
│     7 │            mRNA_BNT_162b2 │
│     8 │                   M_ppi_c │
│     9 │                   M_amp_c │
│    10 │                   M_ump_c │
│    11 │                   M_cmp_c │
│    12 │                   M_gmp_c │
│    13 │           T7RNAP_inactive │
│    14 │      G_BNT_162b2_inactive │
│    15 │ G_BNT_162b2_T7RNAP_closed │
│    16 │   G_BNT_162b2_T7RNAP_open │
└───────┴───────────────────────────┘


#### Setup the constants, feed rates and compositions

In [9]:
# how many chips in parallel?
number_of_chips = 1;

# what fraction of the mRNA degrades?
δ = 0.10;

# MSU split ratio -
θ = 0.90;

# Setup constants for transcription -
L = length(gene_sequence);
K = 0.116; # saturation constant units: μmol/L; Source: ACS Synth. Biol. 2018, 7, 8, 1844–1857 https://doi.org/10.1021/acssynbio.7b00465
v̇ₜ = (90.0)*(60); # units: nt/s; Source: BIND: 111871
u = 0.95; # u-factor; Source: ACS Synth. Biol. 2018, 7, 8, 1844–1857 https://doi.org/10.1021/acssynbio.7b00465

# volume -
V = 100.0*(1/1e6); # liquid reaction volume on each chip units: L

# Stock solution 1: DNA -> feed into splitter (mux) by pump 1
# Composition of the DNA source solution (flows into the chip in stream/channel 1)
G_BNT_162b2 = 1.0;     # gene concentration in stock solution units: μmol/L

# Stock solution 2: PURExpress -> feed into splitter (mux) by pump 2
# Composition of PURExpress (flows into the chip in stream/channel 2)
T7RNAP = 100.0;          # concentration in PURExpress units: μmol/L
M_atp_c = 100*(1e6/1e3); # concentration in PURExpress units: μmol/L
M_utp_c = 100*(1e6/1e3); # concentration in PURExpress units: μmol/L
M_ctp_c = 100*(1e6/1e3); # concentration in PURExpress units: μmol/L
M_gtp_c = 100*(1e6/1e3); # concentration in PURExpress units: μmol/L

# Volumetric flow rates from the pump *into* the splitter unit - our base case will 1 ml/min into the chips, thus, we need
# to scale by the number of chips
V̇₁ = number_of_chips*1000.0*(1/1e6); # volumetric flow rate of syringe pump 1 (this goes into splitter 1) units: L/min
V̇₂ = number_of_chips*1000.0*(1/1e6); # volumetric flow rate of syringe pump 2 (this goes into splitter 2) units: L/min

# stuff needed for later -
F̂₁ = V̇₁/(V̇₁+V̇₂); # do not change me
F̂₂ = V̇₂/(V̇₁+V̇₂); # do not change me

# stuff for the tables -
current_table_counter = 0; # do not change me

#### Specify the inputs streams

##### a) Specify the composition of feed stream 1
By default, this stream contains only the gene encoding the mRNA BNT-162b2 product. The mainstream is split into sub-streams that are fed into each chip.

In [10]:
# setup feed compostions for feed stream 1
ṅ₁ = zeros(ℳ); # default is zero, correct specific values -
ṅ₁[1] = G_BNT_162b2*V̇₁*(1/number_of_chips); # units: μmol/min

##### b) Specify the composition of feed stream 2
By default, feed stream 2 contains the [PURExpress](https://www.neb.com/products/e6800-purexpress-invitro-protein-synthesis-kit#Product%20Information), which has everything we need to make our mRNA product of interest _expect_ the linear DNA. The mainstream is split into sub-streams that are fed into each chip.

In [11]:
# setup feed compostions for feed stream 2
ṅ₂ = zeros(ℳ); # default is zero, then correct specific values -
ṅ₂[2] = T7RNAP*V̇₂*(1/number_of_chips);  # units: μmol/min
ṅ₂[3] = M_atp_c*V̇₂*(1/number_of_chips); # units: μmol/min
ṅ₂[4] = M_utp_c*V̇₂*(1/number_of_chips); # units: μmol/min
ṅ₂[5] = M_ctp_c*V̇₂*(1/number_of_chips); # units: μmol/min
ṅ₂[6] = M_gtp_c*V̇₂*(1/number_of_chips); # units: μmol/min

### Flux balance analysis setup
* Describe the flux balance analysis method. 
* How did your group update the default species and flux bounds to model ths process?
* What objective function did you choose?

#### Species bounds

In [12]:
# species bounds array -
species_bounds_array = [-(ṅ₁ .+ ṅ₂) 10000.0*ones(ℳ,1)];

#### Reaction bounds

In [13]:
# initialize -
flux_bounds_array = zeros(ℛ,2);
flux_bounds_array[:,2] .= 1000.0; # large default upper bound

# Get the RNAP and GENE concentration -
Rₜ = T7RNAP*F̂₂;        # effective RNAP concentratation on the chip for the bounds units: μmol/L
GENE = G_BNT_162b2*F̂₁ # effective GENE concentratation on the chip for the bounds units: μmol/L

# Setup bounds for transcription -
flux_bounds_array[3,:] .= Rₜ*(v̇ₜ/L)*u*(GENE/(K+GENE))*V; # equality constraint

# Setup bound for degradation (lower bound)
flux_bounds_array[4,1] = δ*flux_bounds_array[3,1];

In [14]:
# initialize -
reaction_index_table_data = Array{Any,2}(undef, ℛ, 4);

# build table -
for i ∈ 1:ℛ
    reaction_index_table_data[i,1] = i;
    reaction_index_table_data[i,2] = list_of_reactions[i];
    reaction_index_table_data[i,3] = flux_bounds_array[i,1];
    reaction_index_table_data[i,4] = flux_bounds_array[i,2];
end

# setup title string -
reaction_table_title = "Table $(current_table_counter+=1): Reaction index-name mapping table."

# setup header -
reaction_index_header_table = (["Index", "Reaction", "lower bound", "upper bound"], ["", "", "μmol/min", "μmol/min"]);

# build table -
pretty_table(reaction_index_table_data, title=reaction_table_title; header=reaction_index_header_table);

Table 1: Reaction index-name mapping table.
┌───────┬────────────────────────────┬─────────────┬─────────────┐
│ Index │                   Reaction │ lower bound │ upper bound │
│       │                            │    μmol/min │    μmol/min │
├───────┼────────────────────────────┼─────────────┼─────────────┤
│     1 │       TX_BNT_162b2_binding │         0.0 │      1000.0 │
│     2 │          TX_BNT_162b2_open │         0.0 │      1000.0 │
│     3 │    BNT_162b2_transcription │  0.00613802 │  0.00613802 │
│     4 │ mRNA_BNT_162b2_degradation │ 0.000613802 │      1000.0 │
│     5 │          RNAP_deactivation │         0.0 │      1000.0 │
│     6 │          GENE_deactivation │         0.0 │      1000.0 │
└───────┴────────────────────────────┴─────────────┴─────────────┘


#### Objective coefficient array
* What is the objective you used?

In [21]:
# setup the objective coefficient array -
obj_vector = zeros(ℛ);
obj_vector[3] = -1; # why negative?

## Results and Discussion 

### Compute the optimal extent of reaction and exit stream composition per chip

In [16]:
# compute the optimal flux, and then estimate the output on the chip
result = compute_optimal_extent(S, flux_bounds_array, species_bounds_array, obj_vector);

# build a system stream table -
ϵ̇ = result.calculated_flux_array;

# compute the output -
ṅ₃ = (ṅ₁ + ṅ₂) + S*ϵ̇;

# compute the Δ reaction -
Δ = S*ϵ̇;

# get the exit and status flags to make sure all is ok
exit_flag = result.exit_flag;
status_flag = result.status_flag;

In [17]:
system_flux_table_data = Array{Any,2}(undef, ℳ, 6);

# populate the table -
for i ∈ 1:ℳ
    system_flux_table_data[i,1] = list_of_species[i];
    system_flux_table_data[i,2] = i;
    system_flux_table_data[i,3] = ṅ₁[i];
    system_flux_table_data[i,4] = ṅ₂[i];
    system_flux_table_data[i,5] = round(ṅ₃[i], digits=3);
    system_flux_table_data[i,6] = round(Δ[i], digits=3);
end

# title -
species_mft_table_title = "Table $(current_table_counter+=1): Single chip species mole flow rates table; Streams 1 and 2 are inputs, stream 3 is the chip output. 
The solver returned the exit flag = $(exit_flag) and status_flag = $(status_flag)"

# header -
state_table_header = (
    ["Species", "index i", "ṅ₁,ᵢ", "ṅ₂,ᵢ", "ṅ₃,ᵢ", "Δ"], 
    ["","","(μmol/min)", "(μmol/min)", "(μmol/min)", "(μmol/min)"]
);

# show -
pretty_table(system_flux_table_data, title=species_mft_table_title; header = state_table_header)

Table 2: Single chip species mole flow rates table; Streams 1 and 2 are inputs, stream 3 is the chip output. 
The solver returned the exit flag = 0 and status_flag = 5
┌───────────────────────────┬─────────┬────────────┬────────────┬────────────┬────────────┐
│                   Species │ index i │       ṅ₁,ᵢ │       ṅ₂,ᵢ │       ṅ₃,ᵢ │          Δ │
│                           │         │ (μmol/min) │ (μmol/min) │ (μmol/min) │ (μmol/min) │
├───────────────────────────┼─────────┼────────────┼────────────┼────────────┼────────────┤
│               G_BNT_162b2 │       1 │       0.25 │        0.0 │       0.25 │        0.0 │
│                    T7RNAP │       2 │        0.0 │        0.1 │        0.1 │        0.0 │
│                   M_atp_c │       3 │        0.0 │      100.0 │     95.102 │     -4.898 │
│                   M_utp_c │       4 │        0.0 │      100.0 │     93.837 │     -6.163 │
│                   M_ctp_c │       5 │        0.0 │      100.0 │     93.494 │     -6.506 │
│   

### Design a downstream seperation using Magical Seperator Units (MSU)

In [18]:
# build a downstream seperation process with this number of levels:
number_of_levels = 2; # includes zero

In [19]:
# initialize -
tmp_storage_dict = Dict{Int64, MSULatticeModel}();

# is_product_vector -
is_product_vector = zeros(ℳ);
is_product_vector[7] = 1;

# compute the composition array -
for i ∈ 1:ℳ

    if (is_product_vector[i] == 1)
        msu_lattice_model = build(MSULatticeModel; ṅₒ = (number_of_chips)*ṅ₃[i], L = number_of_levels , u = θ, d = (1 - θ));
    else
        msu_lattice_model = build(MSULatticeModel; ṅₒ = (number_of_chips)*ṅ₃[i], L = number_of_levels , u = (1 - θ), d = θ);
    end
    
    # grab -
    tmp_storage_dict[i] = msu_lattice_model;
end

# grab the leaves -
nodes_dict = build_nodes_dictionary(number_of_levels);
children_dict = build_children_dictionary(nodes_dict);
tree_leaves = nodes_dict[number_of_levels-1];

# build a composition array -
number_of_nodes = length(tree_leaves);
composition_array = Array{Float64,2}(undef, number_of_nodes, ℳ);
for i ∈ 1:ℳ
    data = tmp_storage_dict[i].data;
    for j ∈ 1:number_of_nodes
        composition_array[j,i] = data[tree_leaves[j]]
    end
end

# make a pretty table and show the leaves of the tree -

# initialize -
sep_tree_flow_table_data = Array{Any,2}(undef, ℳ, length(tree_leaves) + 3)
for i ∈ 1:ℳ
    sep_tree_flow_table_data[i,1] = list_of_species[i];
    sep_tree_flow_table_data[i,2] = i;
    sep_tree_flow_table_data[i,3] = (number_of_chips)*ṅ₃[i] # put node 0 in table -
        
    for j ∈ 1:length(tree_leaves)
        sep_tree_flow_table_data[i,3+j] = composition_array[j,i]
    end
end

# labels row -
label_row = Array{String,1}();
push!(label_row,"Species");
push!(label_row,"index i")
push!(label_row,"N0")
for j ∈ 1:length(tree_leaves)
    push!(label_row, "N$(tree_leaves[j])");
end

# units row -
units_row = Array{String,1}();
push!(units_row, "");
push!(units_row, "");
for j ∈ 1:length(tree_leaves)+1
    push!(units_row, "μmol/min");
end

# header -
sep_tree_flow_table_header = (label_row, units_row);

# set title -
title = "Table $(current_table_counter+=1): Magical Seperator Unit (MSU) flow table; N0 denotes the feed while N⋆ denotes the leaves of the tree."

# show -
pretty_table(sep_tree_flow_table_data, title=title; header=sep_tree_flow_table_header)

Table 3: Magical Seperator Unit (MSU) flow table; N0 denotes the feed while N⋆ denotes the leaves of the tree.
┌───────────────────────────┬─────────┬──────────┬─────────────┬────────────┬─────────────┬─────────────┬────────────┬─────────────┐
│                   Species │ index i │       N0 │         N16 │        N17 │         N18 │         N19 │        N20 │         N21 │
│                           │         │ μmol/min │    μmol/min │   μmol/min │    μmol/min │    μmol/min │   μmol/min │    μmol/min │
├───────────────────────────┼─────────┼──────────┼─────────────┼────────────┼─────────────┼─────────────┼────────────┼─────────────┤
│               G_BNT_162b2 │       1 │      5.0 │     5.0e-10 │    4.95e-8 │   4.9005e-6 │  0.00048515 │  0.0480298 │     4.75495 │
│                    T7RNAP │       2 │      2.0 │     2.0e-10 │    1.98e-8 │   1.9602e-6 │  0.00019406 │  0.0192119 │     1.90198 │
│                   M_atp_c │       3 │  1902.04 │  1.90204e-7 │ 1.88302e-5 │  0.00186419 │

In [20]:
# build mol frac composition table -

# construct mol frac array -
mol_frac_array = Array{Float64,2}(undef, ℳ, length(tree_leaves)+1);

# node 0 -
ṅ₃_total = sum(ṅ₃);
for i ∈ 1:ℳ
    mol_frac_array[i,1] = ṅ₃[i]*(1/ṅ₃_total);    
end

# get the sums along rows -
ṅ_total = sum(composition_array,dims = 2);
for node ∈ 1:length(tree_leaves)
    for i ∈ 1:ℳ
        mol_frac_array[i,node+1] = composition_array[node,i]*(1/ṅ_total[node]);
    end
end

# initialize -
sep_tree_mol_frac_table_data = Array{Any,2}(undef, ℳ, length(tree_leaves) + 3)
for i ∈ 1:ℳ
    sep_tree_mol_frac_table_data[i,1] = list_of_species[i];
    sep_tree_mol_frac_table_data[i,2] = i;
    sep_tree_mol_frac_table_data[i,3] = round(mol_frac_array[i,1], digits=4) # put node 0 in table -
        
    for j ∈ 1:length(tree_leaves)
        sep_tree_mol_frac_table_data[i, 3+j] = round(mol_frac_array[i,j+1], digits=4)
    end
end

# labels row -
label_mft_row = Array{String,1}();
push!(label_mft_row,"Species");
push!(label_mft_row,"index i")
push!(label_mft_row,"N0")
for j ∈ 1:length(tree_leaves)
    push!(label_mft_row, "N$(tree_leaves[j])");
end

# units row -
units_mft_row = Array{String,1}();
push!(units_mft_row, "");
push!(units_mft_row, "");
for j ∈ 1:length(tree_leaves)+1
    push!(units_mft_row, "mole frac");
end

# header -
sep_tree_mft_table_header = (label_mft_row, units_mft_row);

# set title -
title_mft = "Table $(current_table_counter+=1): Magical Seperator Unit (MSU) composition table."

# show -
pretty_table(sep_tree_mol_frac_table_data, title=title_mft; header=sep_tree_mft_table_header)

Table 4: Magical Seperator Unit (MSU) composition table.
┌───────────────────────────┬─────────┬───────────┬───────────┬───────────┬───────────┬───────────┬───────────┬───────────┐
│                   Species │ index i │        N0 │       N16 │       N17 │       N18 │       N19 │       N20 │       N21 │
│                           │         │ mole frac │ mole frac │ mole frac │ mole frac │ mole frac │ mole frac │ mole frac │
├───────────────────────────┼─────────┼───────────┼───────────┼───────────┼───────────┼───────────┼───────────┼───────────┤
│               G_BNT_162b2 │       1 │    0.0006 │       0.0 │       0.0 │    0.0006 │    0.0006 │    0.0006 │    0.0006 │
│                    T7RNAP │       2 │    0.0002 │       0.0 │       0.0 │    0.0002 │    0.0002 │    0.0002 │    0.0002 │
│                   M_atp_c │       3 │     0.236 │       0.0 │    0.0165 │    0.2357 │     0.236 │     0.236 │     0.236 │
│                   M_utp_c │       4 │    0.2329 │       0.0 │    0.0163 │

### Financial analysis

* Compute the [Net Present Value](https://varnerlab.github.io/ENGRI-1120-IntroToChemE-Book/chapter-1-dir/money-balances.html#net-present-value-npv) of your design using the cost spreadsheet on canvas.
* What price for the mRNA product do we need to set to have a NPV equal to zero?
* How could you improve the financial performance of your design?

## Summary and conclusions
* Fill me in, see rubric.

## References and Additional Resources
* Put any references are additional citations here